In [42]:
import random 
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()
lemme = WordNetLemmatizer()
stop_words_nltk = set(stopwords.words('english'))

def process(text):
    text = text.lower()
    text = re.sub('\W+', ' ', text)
    text = [x for x in [lemme.lemmatize(w) for w in text.split()] if x not in stop_words_nltk]
    return ' '.join(text)

def prepare_main_dataset():
    df = pd.read_csv('/home/ifte-home/Documents/mental_health/suicide/CSSRS/500_Reddit_users_posts_labels.csv')
    df['Post'] = df.Post.apply(lambda sample: process(sample))
    df=df.dropna()
    df.reset_index()
    df['code'] = Le.fit_transform(df['Label'])
    return df

CSSR_DIR = '/home/ifte-home/Documents/mental_health/suicide/CSSRS/'
CSSR_FILES = ['suicidal_indicator.csv', 'suicidal_attempt.csv', 'suicidal_behavior.csv', 
              'suicidal_ideation.csv']
CSSR_CAT = ['Indicator', 'Attempt','Behavior','Ideation']

def generate_dataset(classes, res):
    data = []
    category = []
    for sample in res:
        data.append(''.join(sample))
        category.append(classes)
    return pd.DataFrame(zip(data, category), columns=['Post', 'Label'])

def generate_samples(sentence_num):
    index = 0
    dataset = pd.DataFrame()
    size = 1
    
    for items in CSSR_FILES:
        frame = pd.read_csv(CSSR_DIR + items, index_col=0)
        col_name = list(frame.columns)
        # Making the class evenly distributed
#         if index == 0:
#             size = 4
#         elif index == 1:
#             size = 52
#         elif index == 3:
#             size = 14
#         else:
#             size = 45
        for k in range(size):
            temp = [col_name[items:items + sentence_num] 
                    for items in range(0, len(col_name), sentence_num)]
            dataset = pd.concat([dataset, generate_dataset(CSSR_CAT[index], temp)])
            random.shuffle(col_name)
        index += 1
    return dataset.sample(frac=1)


def generate_dataframe(sent_size):
    x = generate_samples(sent_size)
    x = pd.concat([prepare_main_dataset(), x])
    x = x[['Post','Label','code']]
    x['code'] = Le.fit_transform(x['Label'])
    return x

In [43]:
generate_dataframe(30)

,Post,Label,code
0,viable option youll leaving wife behind youd p...,Supportive,4
1,hard appreciate notion could meet someone else...,Ideation,2
2,hi last night wa sitting ledge window contempl...,Behavior,1
3,tried kill self failed badly cause moment want...,Attempt,0
4,hi nem3030 sort thing enjoy personally always ...,Ideation,2
...,...,...,...
7,Anaplastic astrocytoma of central nervous sys...,Indicator,3
37,Neonatal staphylococcal infection of skin Per...,Indicator,3
14,Cannabis-induced anxiety disorder Psychoactiv...,Indicator,3
48,Addicted to liquor alcohol addiction alcohol ...,Indicator,3
